In [ ]:
import time

In [ ]:
import random #for randomization of ship placement and other things
import numpy as np #for various array operations
import time

number_of_ships = 1 #number of ships that will be initialized on the board
ship_len = 4 #length of every ship. this will need to be replaced for varying length ships
battleship_array_len = 10 #dimensions of the battleship board. 5 = 5x5 board

battleship_array = [] #array representing the battleship board that will be guessed on
battleship_cheat = [] #'cheat sheet' array containing the positions of all ships on the board

shipdict = {
    "Carrier": 5,
    "Battleship": 4,
    "Cruiser": 3,
    "Submarine": 3,
    "Destroyer": 2
}

symboldict = {
    "Carrier": 'C',
    "Battleship": 'B',
    "Cruiser": 'R',
    "Submarine": 'S',
    "Destroyer": 'D'
}


ship_hit_count = {
    "Carrier": 0,
    "Battleship": 0,
    "Cruiser": 0,
    "Submarine": 0,
    "Destroyer": 0
}

ship_sunk = {
    "Carrier": False,
    "Battleship": False,
    "Cruiser": False,
    "Submarine": False,
    "Destroyer": False
}

number_of_ships_sunk = 0

def reset_ship_arrays():
    global ship_hit_count
    global ship_sunk
    global number_of_ships_sunk
    
    for ship in ship_hit_count:
        ship_hit_count[ship] = 0
        
    for ship in ship_hit_count:
        ship_sunk[ship] = False
        
    number_of_ships_sunk = 0



def initialize_arrays(static_answer = False): #randomize battleship placement in 1d cheat array, make guess array
    global battleship_array
    global battleship_cheat  
    battleship_array = []
    battleship_cheat = []
    #make battleship guess array   
    for i in range(battleship_array_len+1):
        row = []
        if i == battleship_array_len:
            for j in range(battleship_array_len):
                row.append(str(j))
        else:
            for j in range(battleship_array_len+1):
                if j == battleship_array_len:
                    row.append(str(i))
                else:
                    row.append('?') #board spaces unknown to attacking player at beginning of game
        battleship_array.append(row)
        
    if static_answer == True:
        random.seed(a=2)

    #make battleship answer array
    for i in range(battleship_array_len+1):
        row = []
        if i == battleship_array_len:
            for j in range(battleship_array_len):
                row.append(str(j))
        else:
            for j in range(battleship_array_len+1):
                if j == battleship_array_len:
                    row.append(str(i))
                else:
                    row.append('x')
        battleship_cheat.append(row)
                
    #place ships in answer array    
    for ship in shipdict:
        ship_len = shipdict[ship]
         
        orientation = random.randint(0, 1) #0 is horizontal (right) 1 is vertical (down)
        if orientation == 0: #horizontal
            #check for overlap
            overlap = True
            while overlap:
                ship_placement_x = random.randint(0, battleship_array_len - 1)
                ship_placement_y = random.randint(0, battleship_array_len - ship_len)
                clearnum = 0
                for i in range(ship_len):
                    if battleship_cheat[ship_placement_x][ship_placement_y + i] == 'x':
                        clearnum += 1
                    if clearnum == ship_len:
                        overlap = False
                
            ship_letter = symboldict[ship]
            #place left to right
            for i in range(ship_len):
                battleship_cheat[ship_placement_x][ship_placement_y + i] = ship_letter
                
        if orientation == 1: #vertical
            #check for overlap
            overlap = True
            while overlap:
                ship_placement_x = random.randint(0, battleship_array_len - ship_len)
                ship_placement_y = random.randint(0, battleship_array_len - 1)
                clearnum = 0
                for i in range(ship_len):
                    if battleship_cheat[ship_placement_x + i][ship_placement_y] == 'x':
                        clearnum += 1
                    if clearnum == ship_len:
                        overlap = False
                
            ship_letter = symboldict[ship]
            #place left to right
            for i in range(ship_len):
                battleship_cheat[ship_placement_x + i][ship_placement_y] = ship_letter
                          
initialize_arrays()

def print_array(array): #prints a battleship array out in ascii format
    print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
      for row in array]))
    print() 

print_array(battleship_array)
print_array(battleship_cheat)

In [ ]:
repeat_penalty = -1 #code corresponding to calling an already called space. obsolete since repeat hits were removed as an option
miss_penalty = 0 #code corresponding to missing on a call
hit_reward = 1 #code corresponding to hitting a ship on a call

sink_bonus = 2 #code corresponding to sinking a ship on a call. the entire system will have to be reworked with multiple ships

result_dict = {
  repeat_penalty: "already called",
  miss_penalty: "miss",
  hit_reward: "hit",  
}

result_reward_dict = { #q-table rewards/penalties associated with each possible result of an action. 
  repeat_penalty: 0,
  miss_penalty: 0,
  hit_reward: 10,  
} 

hit_dict = { 
  True: "hit",
  False: "miss"
} 

def check_guess(guess_x, guess_y): #assigns q-table reward values to various results of actions
    #will definately need to be reworked once the board becomes more complex
    global battleship_array
    global battleship_cheat 
    global number_of_ships_sunk
    
    if battleship_cheat[guess_x][guess_y] == 'x' and battleship_array[guess_x][guess_y] == '?': #miss
        battleship_array[guess_x][guess_y] = 'x'
        return miss_penalty
        
    if battleship_cheat[guess_x][guess_y] == 'x' and battleship_array[guess_x][guess_y] == 'x': #already called miss
        return repeat_penalty
        
    if battleship_cheat[guess_x][guess_y] != 'x' and battleship_array[guess_x][guess_y] == '?': #hit
        #find ship name based on character
        hit_ship = ""
        ship_letter =''
        for ship, letter in symboldict.items():
            if letter == battleship_cheat[guess_x][guess_y]:
                hit_ship = ship
                ship_letter = letter
        ship_hit_count[hit_ship] += 1 #one more hit on this ship
        if ship_hit_count[hit_ship] == shipdict[hit_ship]: #ship has been sunk
            for i in range(battleship_array_len):
                for j in range(battleship_array_len):
                    if battleship_cheat[i][j] == ship_letter:
                        battleship_array[i][j] = ship_letter  
            ship_sunk[hit_ship] = True
            number_of_ships_sunk += 1
        else:
            battleship_array[guess_x][guess_y] = 'O'
        return hit_reward
        
    if battleship_cheat[guess_x][guess_y] != 'x' and battleship_array[guess_x][guess_y] != 'x': #already called hit
        return repeat_penalty
        
    return -1000 #array mismatch or something went really wrong
        
print_guess_results = True #prints result of guesses made (hit, miss)
print_arrays = True #prints the board and cheat arrays  
    
def make_guesses(): #plays out a solitaire game of battleship randomly guessing coords
    total_hits = 0 #total number of hits currently on board
    turn = 0 #turn counter
    turn_limit = 100
    global battleship_array_len
    global ship_len
    global battleship_array
    global battleship_cheat
    already_called = [] #keeps track of coords that have already been called
    
    initialize_arrays()
    reset_ship_arrays()
    
    if print_arrays: #starting state. print out initial arrays
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    while (not all(ship_sunk.values())) and (turn < turn_limit): #what happens on each turn of the game
        turn += 1
        
        #make random guess
        valid_guess = False
        while not valid_guess: #make random guess
            guess_x = random.randint(0, battleship_array_len - 1)
            guess_y = random.randint(0, battleship_array_len - 1)
            if [guess_x, guess_y] not in already_called: #make sure it is not a repeat guess
                already_called.append([guess_x, guess_y])
                valid_guess = True
        
        
        #check result of guess
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
        
        if print_arrays: #resulting arrays after guess
            print("Turn", turn)
            print_array(battleship_array)
            print("Cheat Sheet:")
            print_array(battleship_cheat) 
            print("-------")
        if result_dict[result] == "hit": #hit
            total_hits += 1 
    return turn
    
turns = make_guesses()
print("Finished in", turns, "turns.")


In [ ]:
%%time
#20000 runs of randomly guessing

print_guess_results = False
print_arrays = False
runs = 1000
turn_count_array = [] #keep track on turns taken on every run
for i in range(runs):
    if (i+1) % 100 == 0:
        print("Run " , i+1, " of " , runs)
    #initialize_arrays()
    turn_count = make_guesses() #play one game
    turn_count_array.append(turn_count) #append to the turn count array
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Randomly Guessing")
print(runs, "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)
#print((turn_count_array))

In [ ]:
def action_to_coord(action): #converts a number to the corresponding coordinate (e.g. '5 = (0, 4)', '6 = (1, 0)' (array len 5))
    #janky way to do this, and will be obsolete when the q-table gets put into a better format
    action_num = action
    guess_x = 0
    guess_y = 0
    
    while action_num > 0:
        if action_num >= battleship_array_len: #counting y coord
            guess_x += 1
            action_num -= battleship_array_len
            
        else: #counting x coord
            guess_y += 1
            action_num -= 1
    return guess_x, guess_y


In [ ]:
#Define hyperparameters for RL 
runs = 1000000
learning_rate = 0.8 #currently unused
max_moves = 100
gamma = 0.95 # discounting rate, currently unused

epsilon = 1 #controls explore vs exploit
max_epsilon = 1
min_epsilon = 0.01

decay_rate = 0.00001#used to decrement epsilon
#decay_rate = 0.0000025

hit_history_size = 2

In [ ]:
%%time
#many runs of learning/training - change number of runs in above block

print_guess_results = False
print_arrays = False
turn_count_array = []
global hit
hit = False #true if a ship has been hit on a given turn
global sunk
sunk = False #true if a ship has been sunk on a given turn

#intialize Q-Table 
qtable = {}
qtable["start"] = np.zeros((battleship_array_len, battleship_array_len))


start_time = time.time()

for run in range(runs):
    if (run+1) % 1000 == 0:
        turn_mean = np.mean(turn_count_array[-1000:])
        print("Run " , run+1, " of " , runs, "Epsilon=", epsilon, "Turns:", turn, "Recent_mean:", turn_mean)
    global already_called
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    
    turn = 0
    state = "start" #start
    total_hits = 0  
    already_called = []
    hit_list = []
    
    
    reset_ship_arrays()
    
    
    while (not all(ship_sunk.values())) and (turn < max_moves): #each turn, check if the game should end (needs rework)
        
    #for turn in range(max_moves):
        # choose an action in the current state (explore or exploit)
        choose_mode = random.uniform(0,1)
        
        
        valid_guess = False
        # if choice > epsilon, --> exploitation (take biggest Q value for state)
        if choose_mode > epsilon:
        #if True:
        #find max value in the Q-Table in the current state and check if its been called
            qmax = float('-inf')
            for i in range(0, battleship_array_len):
                for j in range(0, battleship_array_len):
                    if qtable[state][i][j] > qmax and [i, j] not in already_called:
                        qmax = qtable[state][i][j]
                        guess_x, guess_y = i, j
            already_called.append([guess_x, guess_y])
 

        #else random choice --> exploration
        else:
            while not valid_guess:
                guess_x, guess_y = (random.randint(0, battleship_array_len-1),random.randint(0, battleship_array_len-1)) #action possibilities are the board size squared
                if [guess_x, guess_y] not in already_called:
                    already_called.append([guess_x, guess_y])
                    valid_guess = True
                #print("Explore")
        #take action and update the states and rewards
        """
        # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
        # qtable[new_state,:] : all the actions we can take from new state
        qtable[state, action] = qtable[state, action] + learning_rate * (reward + gamma * np.max(qtable[new_state, :]) - qtable[state, action])
        """
        
        #record num ships sunk before this shot for comparison latter
        before_sunk = number_of_ships_sunk
        
        #apply the shot
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
 
        #record ships sunk to compare and see if a ship was sunk from this shot
        after_sunk = number_of_ships_sunk
        
        #determine if a ship has been sunk this turn
        if (after_sunk - before_sunk) != 0:
            sunk = True
        else:
            sunk = False
        
        #if result == hit_reward and total_hits == number_of_ships * ship_len - 1: #sunk ship, more reward
        if result == hit_reward: #comment this out if you are using a miss penalty
            qtable[state][guess_x][guess_y] += result_reward_dict[result] + (sink_bonus * (after_sunk - before_sunk))
        
        #update to next state
        if result_dict[result] == "hit":
            hit = True
            total_hits += 1  
            hit_to_list = str(guess_x) + ", " + str(guess_y)
            hit_list.insert(0, hit_to_list) #add the hit to the front of the list
            hit_list = hit_list[0:hit_history_size] #delete oldest hit
            
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if hit is not an uncalled hit
                    del hit_list[i]
            ###
            
            #state = tuple([(str(hit_list))])
            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships hunk
        else:
            hit = False
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if hit is not an uncalled hit
                    del hit_list[i]  
            ###
            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships hunk
            
        #print(state)
        #add state to Q-Table if it doesn't exist
        if state not in qtable: #and result == hit_reward:
            qtable[state] = np.zeros((battleship_array_len,battleship_array_len))


        turn += 1 
        
    #reduce epsilon (less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*run)
    #print("Total hits:", total_hits)
    turn_count_array.append(turn) #keep track of turns taken in each game

                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print(runs, "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)
#print(hit_list)


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
for key in qtable:
    print(key)
    print(qtable[key])

In [ ]:
%%time
#testing loop
#20000 runs of testing
#same as above loop, but we only exploit (use Q-table)

print_guess_results = False
print_arrays = False
turn_count_array = []
test_runs = 20000

for i in range(test_runs):
    if (i+1) % 1000 == 0:
        print("Run " , i+1, " of " , test_runs)
    global qtable
    global hit
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    
    turn = 0
    state = "start" #start
    total_hits = 0
    already_called = []
    hit_list = []
    
    reset_ship_arrays()

    
    while (not all(ship_sunk.values())) and (turn < max_moves):


        if state in qtable: #always exploit (use highest Q-table values) during testing if state exists
        #find max value in the Q-Table in the current state and check if its been called
            qmax = float('-inf')
            for i in range(0, battleship_array_len):
                for j in range(0, battleship_array_len):
                    if qtable[state][i][j] > qmax and [i, j] not in already_called:
                        qmax = qtable[state][i][j]
                        guess_x, guess_y = i, j
            already_called.append([guess_x, guess_y])
        else: #randomly guess if in new state
            valid_guess = False
            #print(state + " not in qTable")
            while not valid_guess:
                guess_x, guess_y = (random.randint(0, battleship_array_len-1),random.randint(0, battleship_array_len-1)) #action possibilities are the board size squared
                if [guess_x, guess_y] not in already_called:
                    already_called.append([guess_x, guess_y])
                    valid_guess = True
        
        #check if guess is a hit or miss
        before_sunk = number_of_ships_sunk
        
        #apply the shot
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
 
        #record ships sunk to compare and see if a ship was sunk from this shot
        after_sunk = number_of_ships_sunk
        
        #determine if a ship has been sunk this turn
        if (after_sunk - before_sunk) != 0:
            sunk = True
        else:
            sunk = False


        #update to next state if hit
        if result_dict[result] == "hit":
            total_hits += 1    
            hit = True
            hit_to_list = str(guess_x) + ", " + str(guess_y)
            hit_list.insert(0, hit_to_list) #add the hit to the front of the list
            hit_list = hit_list[0:hit_history_size] #delete oldest hit (index 3)
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if oldest hit is not an uncalled hit
                    del hit_list[i]    
            ###
            
            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships sunk
        else:
            hit = False
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if oldest hit is not an uncalled hit
                    del hit_list[i]   
            ###

            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships sunk
        
#         #add state to Q-Table if it doesn't exist
#         if state not in qtable and result == hit_reward:
#             qtable[state] = np.zeros((battleship_array_len,battleship_array_len))
            
        turn += 1        
    already_called = []        
    turn_count_array.append(turn)
                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Q-table Reinforcement Learning")
print(test_runs, "games")
print("Average turns taken:", turn_mean)
print("Stdev of turns:", turn_std)

In [ ]:
#for key in qtable:
#    print(key)
#    print(qtable[key])

In [ ]:
#see how trained q-table works in one game
print_guess_results = True
print_arrays = True
print_guess_results = False
print_arrays = False
turn_count_array = []
reset_ship_arrays()

for i in range(1): #one game
    global qtable
    initialize_arrays(static_answer = False)
    if print_arrays: #starting state
        print("begin")
        print_array(battleship_array)
        print("Cheat Sheet:")
        print_array(battleship_cheat)
        print("-------")
    
    turn = 0
    state = "start" #start
    total_hits = 0
    already_called = []
    hit_list = []
    
    while (not all(ship_sunk.values())) and (turn < max_moves):
        if state in qtable: #always exploit during testing
            #find max value in the Q-Table in the current state and check if its been called
            qmax = float('-inf')
            for i in range(0, battleship_array_len):
                for j in range(0, battleship_array_len):
                    if qtable[state][i][j] > qmax and [i, j] not in already_called:
                        qmax = qtable[state][i][j]
                        guess_x, guess_y = i, j
            already_called.append([guess_x, guess_y])
        else: #randomly guess if in new state
            valid_guess = False
            print(state + " not in qTable")
            while not valid_guess:
                guess_x, guess_y = (random.randint(0, battleship_array_len-1),random.randint(0, battleship_array_len-1)) #action possibilities are the board size squared
                if [guess_x, guess_y] not in already_called:
                    already_called.append([guess_x, guess_y])
                    valid_guess = True

        #print("State:", state)
        #print("Action:", action)
        
        print("Turn:", turn)
        print_array(battleship_array)
#         print("Cheat Sheet:")
#         print_array(battleship_cheat)
        print("qTable for current state: ")
        print(qtable[state])
        
        #check if guess is a hit or miss
        before_sunk = number_of_ships_sunk
        
        #apply the shot
        result = check_guess(guess_x, guess_y) #hit, miss, or already called
 
        #record ships sunk to compare and see if a ship was sunk from this shot
        after_sunk = number_of_ships_sunk
        
        #determine if a ship has been sunk this turn
        if (after_sunk - before_sunk) != 0:
            sunk = True
        else:
            sunk = False
        
        print("Next guess:", guess_x, guess_y)
        
        

        #update to next state
        if result_dict[result] == "hit":
            hit = True
            total_hits += 1   
            hit_to_list = str(guess_x) + ", " + str(guess_y)
            hit_list.insert(0, hit_to_list) #add the hit to the front of the list
            hit_list = hit_list[0:hit_history_size] #delete oldest hit
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if oldest hit is not an uncalled hit
                    del hit_list[i]    
            ###
            
            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships sunk
        else:
            hit = False
            ### delete sunk ships from hit list
            for i in range(len(hit_list) - 1, -1, -1):
                if (battleship_array[int(hit_list[i][0])][int(hit_list[i][3])]) != "O": #if oldest hit is not an uncalled hit
                    del hit_list[i]   
            ###

            state = tuple([tuple(hit_list), number_of_ships_sunk]) #last 2 hits, number of ships sunk
            
        #add state to Q-Table if it doesn't exist
        if state not in qtable and result == hit_reward:
            qtable[state] = np.zeros((battleship_array_len,battleship_array_len))
            
            
        turn += 1   
        
    already_called = []        
    #reduce epsilon (less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*i)
    #print("Total hits:", total_hits)
    turn_count_array.append(turn)
    
print("victory")
print_array(battleship_array)
print("-------")                   
turn_mean = np.mean(turn_count_array)
turn_std = np.std(turn_count_array)
print("Strategy: Q-table Reinforcement Learning")
print(int(1), "games")
print("Turns taken:", turn_mean)


In [ ]:
### human player
print_guess_results = True
print_arrays = True
turn_count_array = []
reset_ship_arrays()
human = False
if human:
    for i in range(1): #one game
        global qtable
        initialize_arrays(static_answer = False)
        if print_arrays: #starting state
            print("begin")
            print_array(battleship_array)
            print("-------")
        turn = 0
        total_hits = 0
        already_called = []
        while (not all(ship_sunk.values())) and (turn < max_moves):
            print("Turn:", turn)
            print_array(battleship_array)
            guess_y = int(input("Enter x-coord: ")) #prompt user for x-coord guess
            guess_x = int(input("Enter y-coord: ")) #prompt user for y-coord guess
            result = check_guess(guess_x, guess_y) #hit, miss, or already called
            print(result_dict[result]) #tell user what the result of their call was
            if result_dict[result] == "hit":
                total_hits += 1
            turn += 1
        already_called = []
        #print("Total hits:", total_hits)
        turn_count_array.append(turn)
    print("victory")
    print_array(battleship_array)
    print("-------")
    turn_mean = np.mean(turn_count_array)
    turn_std = np.std(turn_count_array)
    print("Strategy: Human Player")
    print(int(1), "games")
    print("Turns taken:", turn_mean)

